 # TUTORIAL - sample project

https://machinelearningmastery.com/object-recognition-with-deep-learning/

https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/

https://towardsdatascience.com/object-detection-with-neural-networks-a4e2c46b4491

Book; http://neuralnetworksanddeeplearning.com/chap6.html

https://www.pyimagesearch.com/2020/10/05/object-detection-bounding-box-regression-with-keras-tensorflow-and-deep-learning/


https://blog.paperspace.com/mask-r-cnn-tensorflow-2-0-keras/

Useful ones:

https://blog.paperspace.com/mask-r-cnn-in-tensorflow-2-0/

https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/

Using Region-Based CNN, R-CNN. Best so far is called the 'Mask R-CNN'. An upgrade to R-CNN, faster R-CNN, YOLO, etc.

In [1]:
import tensorflow

In [2]:
import mrcnn

from mrcnn.config import Config
from mrcnn import model
from mrcnn import visualize
from mrcnn import utils

import numpy as np
import tensorflow as tf
import keras
import colorsys
import argparse
import imutils
import random
import cv2
import os

import xml.etree.ElementTree

from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline

print(tf.__version__)
print(keras.__version__)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)


from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

2.4.1
2.4.3


In [3]:
# # Failed Attempts

# # TF 1.14 implementation - Medium article (doesn't really work due to some log issue)
# class myMaskRCNNConfig(Config):
#     NAME = 'MaskRCNN_inference'
#     GPU_COUNT = 1
#     IMAGES_PER_GPU = 1
#     NUM_CLASSES = 1+80


# config = myMaskRCNNConfig()
# model = modellib.MaskRCNN(mode='inference', config=config, model_dir='./')

# # TF 2.0+ implementation - Does not work - too much other issues

# CLASS_NAMES = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
# # same classes as before (81)

# class simpleConfig(Config):
#     NAME = 'coco_inference'
#     GPU_COUNT = 1
#     IMAGES_PER_GPU = 1

#     NUM_CLASSES = len(CLASS_NAMES)

# model = model.MaskRCNN(mode='inference', config=simpleConfig(), model_dir=os.getcwd())

In [4]:
# 1. Prepare the model configuration parameters

class SimpleConfig(Config):
    NAME = 'coco_inference'
    
    # Used to calculate batch size
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    NUM_CLASSES = 81

In [5]:
# 2. Building the Mask RCNN Model architecture

model = model.MaskRCNN(mode='inference',
                       config=SimpleConfig(),
                       model_dir=os.getcwd())

model.keras_model.summary()

AttributeError: in user code:

    /Users/OAA/PycharmProjects/Mask_RCNN/mrcnn/model.py:390 call  *
        roi_level = log2_graph(tf.sqrt(h * w) / (224.0 / tf.sqrt(image_area)))
    /Users/OAA/PycharmProjects/Mask_RCNN/mrcnn/model.py:341 log2_graph  *
        return tf.log(x) / tf.log(2.0)

    AttributeError: module 'tensorflow' has no attribute 'log'


In [ ]:
# 3. Load the Model Weights

model.load_weights(filepath='mask_rcnn_coco.h5', by_name=True)

In [ ]:
# 4. Read an input image

image = cv2.imread('zebra.jpeg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Reordering the color channels to RGB, rather than BGR

In [ ]:
# 5. Detect objects

r = model.detect(images=[image], verbose=0)

r = r[0]
r.keys()

In [ ]:
# 6. Visualize

CLASS_NAMES = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# r = r[0]

visualize.display_instances(image=image, 
                            boxes=r['rois'],
                            masks=r['masks'],
                            class_ids=r['class_ids'],
                            class_names=CLASS_NAMES,
                            scores=r['scores'])

# Custom Training Set

1. load_dataset - directory of images/annots
2. load_mask - loads masks - accepts image ID (could be anything) and returns masks/class ID for object.
3. extract_boxes - returns coordinates of each bounding box

In [ ]:
import mrcnn

from mrcnn import model
from mrcnn import visualize
from mrcnn import utils

from mrcnn.config import Config
from mrcnn.model import MaskRCNN, load_image_gt, mold_image

import numpy as np
import tensorflow as tf
import keras
import colorsys
import argparse
import imutils
import random
import cv2
import os

from xml.etree import ElementTree

from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline

print(tf.__version__)
print(keras.__version__)

import warnings
# warnings.filterwarnings(action="ignore", message="^internal gelsd")
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)


# from tensorflow.python.util import deprecation
# deprecation._PRINT_DEPRECATION_WARNINGS = False

# Doesnt work
os.environ["TF_CPP_MIN_LOG_LEVEL"]="1"

# Preparing Dataset - KangarooDataset

In [ ]:


# Defining a new class to load the dataset
class KangarooDataset(utils.Dataset):
    
    def load_dataset(self, dataset_dir, is_train=True):
        self.add_class("dataset", 1, "kangaroo") # Adding class - eg. arm - 1, hands - 0, etc

        img_dir = dataset_dir + '/images/'
        ann_dir = dataset_dir + '/annots/'

        for filename in os.listdir(img_dir):
            img_id = filename[:-4]

            if img_id in ['00090']: # Skip 90 cause it's bad
                continue

            if is_train and int(img_id) >= 150: # Training data only until 150
                continue

            if not is_train and int(img_id) < 150: # Test data after 150
                continue

            img_path = img_dir + filename
            ann_path = ann_dir + img_id + '.xml'

            self.add_image('dataset', image_id=img_id, path=img_path, annotation=ann_path)

    # Function to extract bounding boxes from an annotation file
    # Short-cutting development, directly extracting the data with XPath queries
    def extract_boxes(self, filename): 
        tree = ElementTree.parse(filename) # Loading/parsing the annotation

        root = tree.getroot() # retrieving the root element

        boxes = list()
        for box in root.findall('.//bndbox'): # Finding min/max values that define each bounding box
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)

        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

    # Returning one or more masks for photos and classes for each mask
    def load_mask(self, image_id):
        
        # First load the annotation file:
        
        info = self.image_info[image_id] # Retrieving image info dict for image_id - previously defined in load_dataset
        path = info['annotation']
        boxes, w, h = self.extract_boxes(path)
        
        masks = np.zeros([h, w, len(boxes)], dtype='uint8') # Marking all as 0
        # Mask - a 2d array with same dimensions as photograph,
        # all zeros where background, and all ones when it's the object

        # Creating masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            
            class_ids.append(self.class_names.index('kangaroo')) # Marking the bounding box areas as 1
            
        return masks, np.asarray(class_ids, dtype='int32') 
    
    
    # Loading image reference
    def image_reference(self, image_id):
        info = self.image_info(image_id)
        return info['path']

In [ ]:
train_set = KangarooDataset()
train_set.load_dataset(dataset_dir='kangaroo-master', is_train=True)
train_set.prepare()

print(f'Train: {len(train_set.image_ids)}')

In [ ]:
# Test Set

test_set = KangarooDataset()
test_set.load_dataset(dataset_dir='kangaroo-master', is_train=False)
test_set.prepare()

print(f'Test: {len(test_set.image_ids)}')

In [ ]:
# Testing:
id = 24

# Load img
image = train_set.load_image(id)
print(image.shape)

# Load mask
mask, class_ids = train_set.load_mask(id)
print(mask.shape)

In [ ]:
# Plot image
plt.imshow(image)
plt.imshow(mask[:, :, 0], cmap='gray', alpha=0.5)

# Preparing Model + Config

In [ ]:

class KangarooConfig(Config):
    NAME = 'kangaroo_cfg'
    
    NUM_CLASSES = 2 #only 2 - kangaroo + BG(background)
    
    STEPS_PER_EPOCH = 131
    
config = KangarooConfig()

In [ ]:
# Model Creation


kang_model = mrcnn.model.MaskRCNN(mode='training', 
                                  model_dir='./', 
                                  config=config)

In [ ]:

kang_model.load_weights('mask_rcnn_coco.h5',
                        by_name=True,
                        exclude=['mrcnn_class_logits', 'mrcnn_bbox_fc', 'mrcnn_bbox', 'mrcnn_mask'])

# excluded are the ones responsible for class classification, bounding boxes and masks

In [ ]:
kang_model.train(train_set,
                 test_set,
                 learning_rate=config.LEARNING_RATE,
                 epochs=5,
                 layers='heads') # We only train the heads - output layers

In [ ]:
model_path = 'Kangaroo_mask_rcnn.h5'
kan_model.keras_model.save_weights(model_path)

# Evaluating the Model

Positive if IoU is greater than 0.5.

We we do is to enumerate the images in dataset, making a prediction and calculating AP for the prediction. Then get a mean one across all images.

In [ ]:
# Config to making predictions

class PredictionConfig(Config):
    NAME = 'kangaroo_cfg'
    
    NUM_CLASSES = 2
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

pred_config = PredictionConfig()
pred_model = model.MaskRCNN(mode='inference',
                            model_dir='./',
                            config=pred_config)

In [ ]:
# Loading the weights of the final epoch 
pred_model.load_weights('mask_rcnn_kangaroo_cfg_0005.h5', by_name=True)

In [ ]:
def evaluate_model(dataset, model, config):
    APs = list()
    
    for image_id in dataset.image_ids:
        # Load image, bounding boxes/masks for image id
        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, config, image_id, use_mini_mask=False)
        
        # Scaling pixel values of loaded image
        scaled_image = mold_image(image, config)
        # Convert image into one sample
        sample = np.expand_dims(scaled_image, 0)
        
        # Prediction
        yhat = model.detect(sample, verbose=0)
        r = yhat[0] # Results for the first sample
        
        AP, _, _, _ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'])
        APs.append(AP)
        
    mAP = np.mean(APs)
    return mAP

In [ ]:
train_mAP = evaluate_model(train_set, pred_model, pred_config)
print(f"Train mAP: {train_mAP}")

In [ ]:
test_mAP = evaluate_model(test_set, pred_model, pred_config)
print(f"Test mAP: {test_mAP}")

# Making actual Predictions

In [ ]:
# Making predictions on a brand new image

new_img = cv2.imread('kang_1.jpeg')

# General pre-processing
scaled_img = mold_image(new_img, pred_config)
sample = np.expand_dims(scaled_img, 0)

yhat = pred_model.detect(sample, verbose=0)

In [ ]:
# A function, taking a dataset, model, and config, creating a plot of the first 5 photos

def plot_actualVSpredicted(dataset, model, config, n_images=5):
    
    for i in range(n_images):
        img = dataset.load_image(i)
        mask, _ = dataset.load_mask(i)
        
        # Standard Pre-processing
        scaled_img = mold_image(img, pred_config)
        sample = np.expand_dims(scaled_img, 0)
        yhat = pred_model.detect(sample, verbose=0)[0]
        
        plt.figure(figsize=(12, 12))
        # Original
        plt.subplot(n_images, 2, i*2+1) # Defining subplot
        # Plot raw pixel data
        plt.imshow(img)
        plt.title("Actual")
        # Plot masks
        for j in range(mask.shape[2]):
            plt.imshow(mask[:, :, j], cmap='gray', alpha=0.5)

    
        # A second one beside the first, with the first, plot photo again and with bounding boxes in red
        # With bounding boxes
        plt.subplot(n_images, 2, i*2+2)

        plt.imshow(img)
        plt.title('Predicted')
        
        # Plotting each box
        ax=plt.gca()
        for box in yhat['rois']:
            y1, x1, y2, x2 = box
            width, height = x2-x1, y2-y1
            rect = Rectangle((x1, y1), width, height, fill=False, color='blue')

            # Draw the box
            ax.add_patch(rect)
            
    plt.show()

In [ ]:
plot_actualVSpredicted(train_set, pred_model, pred_config)

In [ ]:
plot_actualVSpredicted(test_set, pred_model, pred_config)

In [ ]:
print('x')